In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("car data.csv")

In [3]:
df.dtypes

Car_Name          object
Year               int64
Selling_Price    float64
Present_Price    float64
Kms_Driven         int64
Fuel_Type         object
Seller_Type       object
Transmission      object
Owner              int64
dtype: object

In [4]:
df.isnull().sum()

Car_Name         0
Year             0
Selling_Price    0
Present_Price    0
Kms_Driven       0
Fuel_Type        0
Seller_Type      0
Transmission     0
Owner            0
dtype: int64

In [5]:
#Checking values of each variable

#df["Car_Name"].value_counts()
# df["Year"].value_counts()
# df["Selling_Price"].value_counts()
# df["Present_Price"].value_counts()
# df["Kms_Driven"].value_counts()
df["Fuel_Type"].value_counts()
#df["Seller_Type"].value_counts()
#df["Transmission"].value_counts()
#df["Owner"].value_counts()

Petrol    239
Diesel     60
CNG         2
Name: Fuel_Type, dtype: int64

In [10]:
X=df.drop(["Car_Name"],1)
# X=X.drop(["Owner"],1)
X=X.drop(["Selling_Price"],1)
Y=df["Selling_Price"]

In [11]:
import numpy as np
number=X.select_dtypes(include=np.number)
categorical=X.select_dtypes(exclude=np.number)

In [12]:
categorical=pd.get_dummies(categorical)

In [13]:
X=pd.concat((number,categorical),1)

In [15]:
X.head()

,Year,Present_Price,Kms_Driven,Owner,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Seller_Type_Dealer,Seller_Type_Individual,Transmission_Automatic,Transmission_Manual
0,2014,5.59,27000,0,0,0,1,1,0,0,1
1,2013,9.54,43000,0,0,1,0,1,0,0,1
2,2017,9.85,6900,0,0,0,1,1,0,0,1
3,2011,4.15,5200,0,0,0,1,1,0,0,1
4,2014,6.87,42450,0,0,1,0,1,0,0,1


In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import r2_score

In [19]:
x_train,x_test,y_train,y_test=tts(X,Y,test_size=0.3, random_state=42)

In [20]:
reg=LinearRegression()
reg.fit(x_train,y_train)
y_pred=reg.predict(x_test)
r2_score(y_test,y_pred)

0.875615524620464

In [21]:
from sklearn.feature_selection import RFE

In [59]:
rfe = RFE(reg,9)
rfe = rfe.fit(X,Y)
rfe.support_

array([False,  True,  True, False,  True,  True,  True, False,  True,
        True,  True,  True])

In [60]:
cols=list(X)
cols=pd.Series(cols)
cols[rfe.support_]

1                       Year
2              Present_Price
4                      Owner
5              Fuel_Type_CNG
6           Fuel_Type_Diesel
8         Seller_Type_Dealer
9     Seller_Type_Individual
10    Transmission_Automatic
11       Transmission_Manual
dtype: object

In [61]:
features=["Year","Present_Price","Owner","Fuel_Type_CNG","Fuel_Type_Diesel","Seller_Type_Dealer","Seller_Type_Individual","Transmission_Automatic","Transmission_Manual"]
#features=["Year","Present_Price"]

In [58]:
reg.fit(x_train[features],y_train)
y_pred=reg.predict(x_test[features])
r2_score(y_test,y_pred)

0.8726630442403764

In [43]:
import statsmodels.api as sm
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Selling_Price   R-squared:                       0.883
Model:                            OLS   Adj. R-squared:                  0.879
Method:                 Least Squares   F-statistic:                     274.3
Date:                Mon, 25 Mar 2019   Prob (F-statistic):          5.71e-131
Time:                        14:57:24   Log-Likelihood:                -593.62
No. Observations:                 301   AIC:                             1205.
Df Residuals:                     292   BIC:                             1239.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   -338.3552     37.076     -9.126      0.000    -411.326    -265.384
Year                       0.3931      0.043      9.159      0.000       0.309       0.478
Present_Price              0.4372      0.016     27.355      0.000       0.406       0.469
Kms_Driven             -7.013e-06   3.23e-06     -2.172      0.031   -1.34e-05   -6.57e-07
Owner                     -0.6742      0.423     -1.595      0.112      -1.506       0.158
Fuel_Type_CNG           -113.8109     12.367     -9.203      0.000    -138.151     -89.470
Fuel_Type_Diesel        -111.3410     12.402     -8.978      0.000    -135.749     -86.933
Fuel_Type_Petrol        -113.2033     12.352     -9.165      0.000    -137.513     -88.893
Seller_Type_Dealer      -168.6169     18.546     -9.092      0.000    -205.118    -132.116
Seller_Type_Individual  -169.7383     18.531     -9.160      0.000    -206.210    -133.267
Transmission_Automatic  -168.4535     18.558     -9.077      0.000    -204.977    -131.930
Transmission_Manual     -169.9017     18.520     -9.174      0.000    -206.351    -133.452
==============================================================================
Omnibus:                       95.107   Durbin-Watson:                   1.795
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              696.392
Skew:                           1.075   Prob(JB):                    6.03e-152
Kurtosis:                      10.135   Cond. No.                     6.51e+20
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.04e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [28]:
X = X.drop(["const"],1)

In [29]:
cols = list(X)
p_max = 1
while (len(cols)>0):
    p=[]
    X_1= X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(Y,X_1).fit()
    p = list(model.pvalues.values[1:])
    p = pd.Series(p,index = cols)
    p_max = max(p)
    features_with_p_max = p.idxmax()
    if (p_max>0.05):
        cols.remove(features_with_p_max)
    else:
        break
selected_features = cols

In [30]:
selected_features

['Year',
 'Present_Price',
 'Kms_Driven',
 'Fuel_Type_CNG',
 'Fuel_Type_Diesel',
 'Fuel_Type_Petrol',
 'Seller_Type_Dealer',
 'Seller_Type_Individual',
 'Transmission_Automatic',
 'Transmission_Manual']

In [34]:
features_sm=["Year","Present_Price","Kms_Driven","Fuel_Type_CNG","Fuel_Type_Diesel","Fuel_Type_Petrol","Seller_Type_Dealer","Seller_Type_Individual","Transmission_Automatic","Transmission_Manual"]

In [35]:
reg=LinearRegression()
reg.fit(x_train[features_sm],y_train)
y_pred=reg.predict(x_test[features_sm])
r2_score(y_test,y_pred)

0.8766760433281522